In [1]:
import pandas as pd

In [2]:
## how this data is generated? it is not on  github
df_html = pd.read_parquet("../data/HTML_data.parquet")

In [3]:
df_html

,reference_id,domain_start_id,response_url,response_text
0,23884,3292,https://bianchi-industrial.it/products/linear-...,<!doctypehtml><html class=no-js lang=en-GB><me...
1,65092,1265,https://vilagrancha.com/en/1292-complete-clutc...,<!doctypehtml><html lang=en><meta charset=utf-...


In [4]:
df = df_html.head(50)

In [5]:
df

,reference_id,domain_start_id,response_url,response_text
0,23884,3292,https://bianchi-industrial.it/products/linear-...,<!doctypehtml><html class=no-js lang=en-GB><me...
1,65092,1265,https://vilagrancha.com/en/1292-complete-clutc...,<!doctypehtml><html lang=en><meta charset=utf-...


In [4]:
import os
import requests
import urllib3
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

# Suppress SSL warnings globally
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [7]:
def extract_img_attributes(html):
    """
    Extracts attributes from all <img> tags in the provided HTML content.
    Returns a list of dictionaries, each containing attributes of an <img> tag.
    """
    soup = BeautifulSoup(html, 'lxml')
    img_tags = soup.find_all('img')
    img_data = [img.attrs for img in img_tags]  # List comprehension for brevity
    
    return img_data

In [32]:
def save_combined_html(df, output_file="../data/combined.html"):
    # Combine all response text into one HTML file
    with open(output_file, "w", encoding="utf-8") as file:
        for html_content in df["response_text"]:
            file.write(html_content)
            file.write("\n")  # Separate each HTML content by a newline for readability

In [33]:
def download_images_with_local_path(dict_list, base_url, download_folder="../data/images"):
    """
    Downloads images from URLs in img attributes list and stores them locally.
    Adds the local path of each downloaded image back to the dictionary.
    """
    os.makedirs(download_folder, exist_ok=True)
    default_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    
    for index, img_data in enumerate(dict_list):
        img_url = img_data.get("src")
        
        # Convert relative URLs to absolute URLs
        if img_url and urlparse(img_url).scheme == "":
            img_url = urljoin(base_url, img_url)
            print(f"Converted relative URL to absolute: {img_url}")

        # Replace backslashes with forward slashes
        if img_url:
            img_url = img_url.replace("\\", "/")
        
        # Check if URL is valid and not a data URI
        if img_url and urlparse(img_url).scheme in ["http", "https"]:
            # Generate filename consistent with the original function
            img_name = f"{index}_{os.path.basename(urlparse(img_url).path)}"
            img_path = os.path.join(download_folder, img_name)
            
            # Download the image using requests with SSL verification disabled
            try:
                # Attempt to download with custom User-Agent on the first try
                response = requests.get(img_url, headers=default_headers, stream=True, verify=False)
                response.raise_for_status()  # Raise an error for HTTP issues
                with open(img_path, "wb") as img_file:
                    for chunk in response.iter_content(1024):
                        img_file.write(chunk)
                print(f"Downloaded: {img_name}")
                
                # Add the relative path to the dictionary
                img_data["local_path"] = os.path.relpath(img_path, start=download_folder)
                
            except Exception as e:
                # Print the error and skip to the next item
                print(f"Error downloading {img_url}: {e}")
                
        else:
            # Skip data URIs or invalid URLs
            if img_url and img_url.startswith("data:image"):
                print(f"Skipping item {index} - Data URI found.")
            else:
                print(f"Skipping item {index} - No valid image URL found.")

In [ ]:
# Process each HTML and base URL from the DataFrame
all_images_data = []

for idx, row in df.iterrows():
    html_content = row["response_text"]
    base_url = row["response_url"]
    
    # Extract image attributes for the current HTML page
    img_data = extract_img_attributes(html_content)
    
    # Download images and resolve paths
    download_images_with_local_path(img_data, base_url=base_url)
    
    # Add base URL and image data to the overall list
    for img in img_data:
        img["source_url"] = base_url  # Keep track of the page URL where the image was found
    all_images_data.extend(img_data)

# Optionally, convert all image data to a DataFrame for analysis
img_df = pd.DataFrame(all_images_data)

In [ ]:
def clean_image_folder(folder_path="../data/images"):
    """
    Deletes all files in the specified folder that are not .jpg files or contain 'logo' in the filename (case-insensitive).
    """
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if file is not a .jpg or contains "logo" (case insensitive)
        if not filename.lower().endswith(".jpg") or "logo" in filename.lower():
            try:
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except Exception as e:
                print(f"Error deleting {filename}: {e}")

# Usage
clean_image_folder("../data/images")

# Incorporating it into the app

1. Changing process_html function 

In [1]:
def process_html(html, base_url, model):
    """nahui ne nuzhna -> replace with the download and classify

    Args:
        html (str): html code
        model (MobileViTClassifier): keep this model as it is
    Returns:
       (dict) 
        html_results = {
            "predictions": [],
            "statistics": defaultdict(int)
            }
    """
    html_results = {
        "predictions": [],
        "statistics": defaultdict(int)
    }
    
    img_data = extract_img_attributes(html, base_url) # TODO -> replace the download_images_with_local_path -> adjust the workflow
    # instead of putting single images in the function as an input dump whole list there.
    for img in img_data:
        # First download the image and add local path
        download_images_with_local_path([img], TEMP_IMAGE_DIR)
        
        # Then check if download was successful and local path was added
        if "local_path" in img:
            # Skip non jpg files and logo images since they can't be processed by the model, check jpeg!!!
            if not img["local_path"].lower().endswith(".jpg") or "logo" in img["local_path"].lower():
                continue

            ### from here you need to put this in the donwload & classify function
            prediction = model.predict(img["local_path"])['prediction']
            
            # Store individual prediction
            html_results["predictions"].append({
                "image_path": img["local_path"],
                "predicted_class": prediction
            })
            
            # Update statistics counter
            html_results["statistics"][prediction] += 1
    
    return html_results

2. Changing download_images_with_local_path function

In [ ]:
def extract_img_attributes(html, base_url):
    """
    Parses the HTML to extract attributes of all <img> tags and processes the 'src' attribute.

    Args:
        html (str): The HTML content.
        base_url (str): The base URL to resolve relative paths in 'src' attributes.

    Returns:
        list: A list of dictionaries containing attributes of each <img> tag, with the 'src' modified.
    """
    from urllib.parse import urljoin, urlparse
    from bs4 import BeautifulSoup

    # Parse the HTML content
    soup = BeautifulSoup(html, 'lxml')

    # Find all <img> tags
    img_tags = soup.find_all('img')

    # Initialize list to store each img tag's attributes as dictionaries
    img_data = []

    # Loop through each img tag and extract attributes
    for img in img_tags:
        img_attributes = img.attrs  # Get all attributes of the img tag as a dictionary
        img_url = img_attributes.get("src")  # Get the 'src' attribute
        
        # Convert relative URLs to absolute URLs
        if img_url and urlparse(img_url).scheme == "":
            img_url = urljoin(base_url, img_url)
            print(f"Converted relative URL to absolute: {img_url}")

        # Replace backslashes with forward slashes
        if img_url:
            img_url = img_url.replace("\\", "/")

        # Update the 'src' attribute in the dictionary
        img_attributes["src"] = img_url
        img_data.append(img_attributes)  # Append dictionary to the list

    return img_data

def save_combined_html(df, output_file="../data/combined.html"):
    # Combine all response text into one HTML file
    with open(output_file, "w", encoding="utf-8") as file:
        for html_content in df["response_text"]:
            file.write(html_content)
            file.write("\n")  # Separate each HTML content by a newline for readability

In [ ]:
def download_images_with_local_path(dict_list, download_folder="../data/images"):
    """
    Downloads images from URLs in a list of dictionaries and adds local file paths.
    
    Args:
        dict_list (list): List of dictionaries containing image attributes.
            Each dictionary has keys like 'src', 'alt', 'title', etc.
            Example:
            [
                {
                    'title': 'some title',
                    'alt': 'some alt text',
                    'border': '0', 
                    'src': 'https://example.com/images/logo.png' - URL to the image
                },
                ...
            ]
        download_folder (str): Path where images will be downloaded to.
            Defaults to "../data/images"
            
    Returns:
        None: Modifies the input dictionaries in-place by adding 'local_path' key
    """

    # Ensure the download folder exists
    os.makedirs(download_folder, exist_ok=True)
    default_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    
    # Loop through each dictionary in the list
    for index, img_data in enumerate(dict_list):
        img_url = img_data.get("src")
        
        # Check if URL is valid and not a data URI
        if img_url and urlparse(img_url).scheme in ["http", "https"]:
            # Generate filename consistent with the original function
            img_name = f"{index}_{os.path.basename(urlparse(img_url).path)}"
            img_path = os.path.join(download_folder, img_name)
            
            # Download the image using requests with SSL verification disabled
            try:
                # Attempt to download with custom User-Agent on the first try
                response = requests.get(img_url, headers=default_headers, stream=True, verify=False)
                response.raise_for_status()  # Raise an error for HTTP issues
                with open(img_path, "wb") as img_file:
                    for chunk in response.iter_content(1024):
                        img_file.write(chunk)
                print(f"Downloaded: {img_name}")
                
                # Add the relative path to the dictionary
                img_data["local_path"] = os.path.relpath(img_path, start=download_folder)
                
            except Exception as e:
                # Print the error and skip to the next item
                print(f"Error downloading {img_url}: {e}")
        else:
            # Skip data URIs or invalid URLs
            if img_url and img_url.startswith("data:image"):
                print(f"Skipping item {index} - Data URI found.")
            else:
                print(f"Skipping item {index} - No valid image URL found.")